I set all of the `enc_shape = es` for every instance of the autoencoder. So just set `es=2, es=4`... to get all of the autoencoders.

In [57]:
es = 4

https://github.com/techshot25/Autoencoders/blob/master/simple-autoencoder.ipynb
This simple code shows you how to make an autoencoder using Pytorch. The idea is to bring down the number of dimensions (or reduce the feature space) using neural networks.

The idea is simple, let the neural network learn how to make the encoder and the decoder using the feature space as both the input and the output of the network.

### Import relevant packages

In [58]:
import torch
from torch import nn, optim
import numpy as np
from matplotlib import pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
from sklearn.preprocessing import MinMaxScaler

import pandas as pd

In [59]:
import cython 

import skfda
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from skfda.exploratory.depth import ModifiedBandDepth, IntegratedDepth
from skfda.exploratory.visualization import Boxplot

from skfda.exploratory.visualization.clustering import (
    ClusterMembershipLinesPlot,
    ClusterMembershipPlot,
    ClusterPlot,
)
from skfda.ml.clustering import FuzzyCMeans, KMeans

from skfda.exploratory.visualization import FPCAPlot
from skfda.preprocessing.dim_reduction.feature_extraction import FPCA
from skfda.representation.basis import BSpline, Fourier, Monomial

### Read in data

In [60]:
X = pd.read_csv('Data/ML_GRF_stance_N.csv').values

In [61]:
Y = pd.read_csv('Data/AP_GRF_stance_N.csv').values

In [62]:
Z = pd.read_csv('Data/V_GRF_stance_N.csv').values

### Define functions and build Autoencoder

In [63]:
class Autoencoder(nn.Module):
    """Makes the main denoising auto

    Parameters
    ----------
    in_shape [int] : input shape
    enc_shape [int] : desired encoded shape
    """

    def __init__(self, in_shape, enc_shape):
        super(Autoencoder, self).__init__()
        
        self.encode = nn.Sequential(
            nn.Linear(in_shape, 128),
            nn.ReLU(True),
            nn.Dropout(0.2), #regularization
            nn.Linear(128, 64),
            nn.ReLU(True),
            nn.Dropout(0.2),
            nn.Linear(64, enc_shape),
        )
        
        self.decode = nn.Sequential(
            nn.BatchNorm1d(enc_shape),
            nn.Linear(enc_shape, 64),
            nn.ReLU(True),
            nn.Dropout(0.2),
            nn.Linear(64, 128),
            nn.ReLU(True),
            nn.Dropout(0.2),
            nn.Linear(128, in_shape)
        )
        
    def forward(self, x):
        x = self.encode(x)
        x = self.decode(x)
        return x

In [64]:
def train(model, error, optimizer, n_epochs, x):
    model.train()
    for epoch in range(1, n_epochs + 1):
        optimizer.zero_grad()
        output = model(x)
        loss = error(output, x)
        loss.backward()
        optimizer.step()
        
        if epoch % int(0.1*n_epochs) == 0:
            print(f'epoch {epoch} \t Loss: {loss.item():.4g}')

In [65]:
#Defining a list of grid points
grid_points_100 = list(range(1,101))
grid_points_2990 = list(range(1,2991))

In [66]:

ne = 1000

#### 1. GRF_ML

In [67]:
device_x = ('cuda' if torch.cuda.is_available() else 'cpu')

In [68]:
x = torch.from_numpy(X).to(device_x)

In [69]:
encoder_x = Autoencoder(in_shape=100, enc_shape=es).double().to(device_x)

error_x = nn.MSELoss()

optimizer_x = optim.Adam(encoder_x.parameters())

In [70]:
train(encoder_x, error_x, optimizer_x, ne, x)

epoch 100 	 Loss: 142.8
epoch 200 	 Loss: 120.4
epoch 300 	 Loss: 105.4
epoch 400 	 Loss: 98.73
epoch 500 	 Loss: 94.54
epoch 600 	 Loss: 92.24
epoch 700 	 Loss: 91.05
epoch 800 	 Loss: 89.89
epoch 900 	 Loss: 88.65
epoch 1000 	 Loss: 85.44


In [71]:
with torch.no_grad():
    encoded_x = encoder_x.encode(x)
    decoded_x = encoder_x.decode(encoded_x)
    mse_x = error_x(decoded_x, x).item()
    enc_x = encoded_x.cpu().detach().numpy()
    dec_x = decoded_x.cpu().detach().numpy()

#### 2. GRF_AP

In [72]:
device_y = ('cuda' if torch.cuda.is_available() else 'cpu')

In [73]:
y = torch.from_numpy(Y).to(device_y)

In [74]:
encoder_y = Autoencoder(in_shape=100, enc_shape=es).double().to(device_y)

error_y = nn.MSELoss()

optimizer_y = optim.Adam(encoder_y.parameters())

In [75]:
train(encoder_y, error_y, optimizer_y, ne, y)

epoch 100 	 Loss: 1322
epoch 200 	 Loss: 426.9
epoch 300 	 Loss: 366.5
epoch 400 	 Loss: 353.1
epoch 500 	 Loss: 345.7
epoch 600 	 Loss: 332.5
epoch 700 	 Loss: 324.8
epoch 800 	 Loss: 316
epoch 900 	 Loss: 305.4
epoch 1000 	 Loss: 301.9


In [76]:
with torch.no_grad():
    encoded_y = encoder_y.encode(y)
    decoded_y = encoder_y.decode(encoded_y)
    mse_y = error_y(decoded_y, y).item()
    enc_y = encoded_y.cpu().detach().numpy()
    dec_y = decoded_y.cpu().detach().numpy()

#### 3. GRF_V

In [77]:
device_z = ('cuda' if torch.cuda.is_available() else 'cpu')

In [78]:
z = torch.from_numpy(Z).to(device_z)

In [79]:
encoder_z = Autoencoder(in_shape=100, enc_shape=es).double().to(device_z)

error_z = nn.MSELoss()

optimizer_z = optim.Adam(encoder_z.parameters())

In [80]:
train(encoder_z, error_z, optimizer_z, ne, z)

epoch 100 	 Loss: 3.524e+05
epoch 200 	 Loss: 1.646e+04
epoch 300 	 Loss: 1.297e+04
epoch 400 	 Loss: 1.232e+04
epoch 500 	 Loss: 1.181e+04
epoch 600 	 Loss: 1.15e+04
epoch 700 	 Loss: 1.127e+04
epoch 800 	 Loss: 1.112e+04
epoch 900 	 Loss: 1.089e+04
epoch 1000 	 Loss: 1.101e+04


In [81]:
with torch.no_grad():
    encoded_z = encoder_z.encode(z)
    decoded_z = encoder_z.decode(encoded_z)
    mse_z = error_z(decoded_z, z).item()
    enc_z = encoded_z.cpu().detach().numpy()
    dec_z = decoded_z.cpu().detach().numpy()

### Calculate the MSE and R^2

In [82]:
# Define function for MSE and TSS

def mse(array1,array2):
    diff = array1 - array2
    sqdiff = diff**2
    n = len(sqdiff)
    
    sum_sqdiff = sum(sqdiff)
    return sum_sqdiff/n

def tss(array):
    return sum((array-sum(array)/len(array))**2)

In [83]:
# GRF_ML

ML_GRF_reconst_array = dec_x.flatten()
ML_GRF_array = X.flatten()

In [84]:
mse(ML_GRF_array, ML_GRF_reconst_array)

85.4088292819283

In [85]:
tss(ML_GRF_array)

1021052935.1509883

In [86]:
1 - mse(ML_GRF_array, ML_GRF_reconst_array) / tss(ML_GRF_array)

0.9999999163522024

In [87]:
# GRF_AP

AP_GRF_reconst_array = dec_y.flatten()
AP_GRF_array = Y.flatten()

In [88]:
mse(AP_GRF_array, AP_GRF_reconst_array)

300.81421932975303

In [89]:
tss(AP_GRF_array)

10927261220.839506

In [90]:
1 - mse(AP_GRF_array, AP_GRF_reconst_array) / tss(AP_GRF_array)

0.9999999724712155

In [91]:
# GRF_V

V_GRF_reconst_array = dec_z.flatten()
V_GRF_array = Z.flatten()

In [92]:
mse(V_GRF_array, V_GRF_reconst_array)

10953.056593085577

In [93]:
tss(V_GRF_array)

129722354256.23286

In [94]:
1 - mse(V_GRF_array, V_GRF_reconst_array) / tss(V_GRF_array)

0.9999999155653885